# ETL Process for Spotify API

This notebook is used to pull my listening activitys and songs data from Spotify API and perform ETL process

## Spotify API Access
To fetch data from Spotify, you'll need to go to https://developer.spotify.com/dashboard/login and create Spotify Developer and create an application.

In the application, there are some identifiers you'll need for an OAuth flow:

1. Under **Show Client Secret**:
 - Client ID
 - Client Secret

2. In **Edit Settings**:
 - Redirect URIs (You can add http://localhost:8888/callback if you run on localhost)
 
 
*Note that you will need an ordinary Spotify account to login, create an app, and get the credentials

In [1]:
# Import necessary libs
import pandas as pd
import numpy as np
import datetime

import spotipy
from spotipy.oauth2 import SpotifyOAuth

from sqlalchemy import create_engine

In [2]:
# Insert your Spotify username and credentials rom Spotify Developer
SPOTIFY_CLIENT_ID = "xxx"
SPOTIFY_CLIENT_SECRET = "xxx"
REDIRECT_URL = "xxx"

# Inser your PostgresSQL database configuration
HOST="xxx"
DATABASE="xxx"
USER="xxx"
PASSWORD="xxx"

In [3]:
# Create a Client Authorization flow and get the token
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(
        client_id=SPOTIFY_CLIENT_ID,
        client_secret=SPOTIFY_CLIENT_SECRET,
        redirect_uri=REDIRECT_URL,
        scope="user-read-recently-played"
    ))

## Perform extraction and transformation for each table

### PlayedSong Table - Fact Table



In [4]:
today = datetime.datetime.now()
ndays_ago = today - datetime.timedelta(days=7)  
time = int(ndays_ago.timestamp()) * 1000
# Get user's recently-played-tracks data and
recently_played_songs = sp.current_user_recently_played(after=time)

# Structure the dataframe for PlayedSong Table
songs_dict = {
        "song_id": [],
        "song_name": [],
        "artist_id": [],
        "artist_name": [],
        "played_at": [],
    }

for song in recently_played_songs['items']:
    songs_dict['song_id'].append(song["track"]['id'])
    songs_dict['song_name'].append(song["track"]["name"])
    songs_dict['artist_id'].append(song["track"]["album"]["artists"][0]["id"])
    songs_dict['artist_name'].append(song["track"]["album"]["artists"][0]["name"])
    songs_dict['played_at'].append(song["played_at"])

played_songs_df = pd.DataFrame(songs_dict)
played_songs_df.head()

,song_id,song_name,artist_id,artist_name,played_at
0,2OKo7g3KfmCt3kyLvUAL0g,The Search,6fOMl44jA4Sp5b9PpYCkzz,NF,2021-06-19T11:31:40.341Z
1,0SmQ0UWeiHKQtxUej4Rem6,Tháng Năm,6CGGvCBHWqQ4HXtn5aLhbh,Soobin Hoang Son,2021-06-19T11:26:50.857Z
2,34uOIsoy6ETS2sxOTWBRDF,Trốn Tìm,1LEtM3AleYg1xabW6CRkpi,Đen,2021-06-19T11:23:05.074Z
3,63qZf6349d3ngku9hrzLac,The Playah (feat. SlimV) - Special Performance,6CGGvCBHWqQ4HXtn5aLhbh,Soobin Hoang Son,2021-06-19T11:18:55.526Z
4,2bgTY4UwhfBYhGT4HUYStN,Butter,3Nrfpe0tUJi4K4DXYWgMUX,BTS,2021-06-19T11:11:05.513Z


In [5]:
# Check missing values
if played_songs_df.isnull().values.any():
    raise Exception("Null value found in played songs data")
    
timestamp_df = pd.to_datetime(played_songs_df['played_at'])
# Change to GMT+7:00 timezone
timestamp_df = timestamp_df + datetime.timedelta(hours=-5)
#Set UNIX code for played_at as primary key, since only one song can played at a time
played_songs_table_pk = timestamp_df.astype(np.int64) // 10**6
# Format the timestamp
timestamp_df = timestamp_df.dt.strftime("%m-%d-%Y %H:%M:%S")
played_songs_df['played_at'] = timestamp_df
played_songs_df.insert(0, "played_song_id", played_songs_table_pk)

played_songs_df.head()

,played_song_id,song_id,song_name,artist_id,artist_name,played_at
0,1624084300341,2OKo7g3KfmCt3kyLvUAL0g,The Search,6fOMl44jA4Sp5b9PpYCkzz,NF,06-19-2021 06:31:40
1,1624084010857,0SmQ0UWeiHKQtxUej4Rem6,Tháng Năm,6CGGvCBHWqQ4HXtn5aLhbh,Soobin Hoang Son,06-19-2021 06:26:50
2,1624083785074,34uOIsoy6ETS2sxOTWBRDF,Trốn Tìm,1LEtM3AleYg1xabW6CRkpi,Đen,06-19-2021 06:23:05
3,1624083535526,63qZf6349d3ngku9hrzLac,The Playah (feat. SlimV) - Special Performance,6CGGvCBHWqQ4HXtn5aLhbh,Soobin Hoang Son,06-19-2021 06:18:55
4,1624083065513,2bgTY4UwhfBYhGT4HUYStN,Butter,3Nrfpe0tUJi4K4DXYWgMUX,BTS,06-19-2021 06:11:05


### Song Table - Dimension Table

In [6]:
# Get list of song ID without duplicate 
song_ids_list = played_songs_df['song_id'].unique().tolist()
# Get multiple songs data
songs_data = sp.tracks(tracks=song_ids_list, market=None) # maximum 50 IDs

In [7]:
# Structure dataframe Songs Table
songs_dict = {
    'song_id': [],
    'song_name' : [],
    'artist_id': [],
    'album_id': [],
    'duration_ms': [],
    'popularity': [],
    'external_urls': []
}

for song in songs_data['tracks']:
    songs_dict['song_id'].append(song['id'])
    songs_dict['song_name'].append(song['name'])
    songs_dict['artist_id'].append(song['artists'][0]['id'])
    songs_dict['album_id'].append(song['album']['id'])
    songs_dict['popularity'].append(song['popularity'])
    songs_dict['duration_ms'].append(song['duration_ms'])
    songs_dict['external_urls'].append(song['external_urls']['spotify'])

songs_df = pd.DataFrame(songs_dict)

In [8]:
# Check missing values
if songs_df.isnull().values.any():
    raise Exception("Null value found in songs data")

In [9]:
songs_df.head()

,song_id,song_name,artist_id,album_id,duration_ms,popularity,external_urls
0,2OKo7g3KfmCt3kyLvUAL0g,The Search,6fOMl44jA4Sp5b9PpYCkzz,6w8mGg73sQl4QJEhpDUvpI,248040,73,https://open.spotify.com/track/2OKo7g3KfmCt3ky...
1,0SmQ0UWeiHKQtxUej4Rem6,Tháng Năm,6CGGvCBHWqQ4HXtn5aLhbh,5xduxfrnEQpYdH54ilWMAN,225358,60,https://open.spotify.com/track/0SmQ0UWeiHKQtxU...
2,34uOIsoy6ETS2sxOTWBRDF,Trốn Tìm,1LEtM3AleYg1xabW6CRkpi,1OWSGECq2oKMojwFp0EFvJ,248870,65,https://open.spotify.com/track/34uOIsoy6ETS2sx...
3,63qZf6349d3ngku9hrzLac,The Playah (feat. SlimV) - Special Performance,6CGGvCBHWqQ4HXtn5aLhbh,4437psfF6TPngVcOkFT90M,449577,62,https://open.spotify.com/track/63qZf6349d3ngku...
4,2bgTY4UwhfBYhGT4HUYStN,Butter,3Nrfpe0tUJi4K4DXYWgMUX,1HnJKmB4P6Z8RBdLMWx18w,164441,93,https://open.spotify.com/track/2bgTY4UwhfBYhGT...


### SongFeature - Dimension Table

In [10]:
# Get audio features data
audio_features = sp.audio_features(tracks=song_ids_list)

In [11]:
# Structure dataframe AudioFeature Table
feature_dict = {
        "song_id": [],
        "danceability": [],
        "energy": [],
        "loudness": [],
        "speechiness": [],
        "acousticness": [],
        "instrumentalness": [],
        "liveness": [],
        "valence": [],
        "tempo": [],
}
for song in audio_features:
    feature_dict['song_id'].append(song['id'])
    feature_dict['danceability'].append(song['danceability'])
    feature_dict['energy'].append(song['energy'])
    feature_dict['loudness'].append(song['loudness'])
    feature_dict['speechiness'].append(song['speechiness'])
    feature_dict['acousticness'].append(song['acousticness'])
    feature_dict['instrumentalness'].append(song['instrumentalness'])
    feature_dict['liveness'].append(song['liveness'])
    feature_dict['valence'].append(song['valence'])
    feature_dict['tempo'].append(song['tempo'])

audio_features_df = pd.DataFrame(feature_dict)

In [12]:
# Check missing values
if audio_features_df.isnull().values.any():
    raise Exception("Null value found in song feature data")

In [13]:
audio_features_df.head()

,song_id,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,2OKo7g3KfmCt3kyLvUAL0g,0.789,0.786,-4.788,0.2970,0.59600,0.000000,0.0997,0.390,119.957
1,0SmQ0UWeiHKQtxUej4Rem6,0.670,0.618,-5.467,0.0377,0.12800,0.000000,0.0534,0.251,147.015
2,34uOIsoy6ETS2sxOTWBRDF,0.771,0.287,-17.208,0.0720,0.66900,0.000000,0.1040,0.402,93.041
3,63qZf6349d3ngku9hrzLac,0.572,0.513,-6.078,0.0358,0.14800,0.000001,0.0597,0.528,116.397
4,2bgTY4UwhfBYhGT4HUYStN,0.759,0.459,-5.187,0.0948,0.00323,0.000000,0.0906,0.695,109.997


### Artist - Dimension Table

In [14]:
# Get list of artist ID without duplicate 
artist_ids_list = played_songs_df['artist_id'].unique().tolist()

In [15]:
# Get multiple artist data and structuren datafram for Artist Table
artists_dict = {
    'artist_id': [],
    'artist_name': [],
    'followers': [],
    'popularity': [],
    'external_urls': []
}

for id in artist_ids_list:
    artist = sp.artist(id)
    artists_dict['artist_id'].append(artist['id'])
    artists_dict['artist_name'].append(artist['name'])
    artists_dict['followers'].append(artist['followers']['total'])
    artists_dict['popularity'].append(artist['popularity'])
    artists_dict['external_urls'].append(artist['external_urls']['spotify'])

artists_df = pd.DataFrame(artists_dict)

In [16]:
# Check missing values
if artists_df.isnull().values.any():
    raise Exception("Null value found in artist data")

In [17]:
artists_df.head()

,artist_id,artist_name,followers,popularity,external_urls
0,6fOMl44jA4Sp5b9PpYCkzz,NF,5577673,86,https://open.spotify.com/artist/6fOMl44jA4Sp5b...
1,6CGGvCBHWqQ4HXtn5aLhbh,Soobin Hoang Son,545585,57,https://open.spotify.com/artist/6CGGvCBHWqQ4HX...
2,1LEtM3AleYg1xabW6CRkpi,Đen,1570632,62,https://open.spotify.com/artist/1LEtM3AleYg1xa...
3,3Nrfpe0tUJi4K4DXYWgMUX,BTS,34401936,98,https://open.spotify.com/artist/3Nrfpe0tUJi4K4...
4,5dfZ5uSmzR7VQK0udbAVpf,Sơn Tùng M-TP,2368594,63,https://open.spotify.com/artist/5dfZ5uSmzR7VQK...


### Time - Dimension Table

In [18]:
# Get time data and structure dataframe for Time Table
played_songs_df['played_at'] = pd.to_datetime(played_songs_df['played_at'])
time_df = pd.DataFrame({"start_time": played_songs_df['played_at'],
                        "time": played_songs_df['played_at'].astype(str).str[11:],
                        "day": played_songs_df['played_at'].dt.day,
                        "month": played_songs_df['played_at'].dt.month,
                        "year": played_songs_df['played_at'].dt.year,
                        "weekday": played_songs_df['played_at'].dt.weekday})

time_df.head()

,start_time,time,day,month,year,weekday
0,2021-06-19 06:31:40,06:31:40,19,6,2021,5
1,2021-06-19 06:26:50,06:26:50,19,6,2021,5
2,2021-06-19 06:23:05,06:23:05,19,6,2021,5
3,2021-06-19 06:18:55,06:18:55,19,6,2021,5
4,2021-06-19 06:11:05,06:11:05,19,6,2021,5


## Load transformed data into database

To load the data into the database,first,create a temporary table, and use `.to_sql` method to load data into it. Then, inserting data from temporary tables to base tables to avoid the table's primary key already exists.

In [19]:
# Connect to PostgresSQL database
engine = create_engine('postgresql+psycopg2://{username}:{password}@{host}/{database}'.format(username=USER, 
                                                                                            password=PASSWORD,
                                                                                            host=HOST,
                                                                                            database=DATABASE))
conn = engine.raw_connection()
cur = conn.cursor()

In [20]:
%%time
# Song Table
cur.execute('''
    CREATE TEMP TABLE IF NOT EXISTS temp_song AS 
    SELECT * FROM Song LIMIT 0;
''')
songs_df.to_sql("temp_song", con=engine, if_exists='replace', index=False)

engine.execute('''
    INSERT INTO public.Song
    SELECT * FROM temp_song
    WHERE temp_song.song_id NOT IN (SELECT song_id FROM public.Song);
''')

Wall time: 126 ms


In [21]:
%%time
# Artist Table
cur.execute('''
    CREATE TEMP TABLE IF NOT EXISTS temp_artist AS 
    SELECT * FROM Artist LIMIT 0;
''')
artists_df.to_sql("temp_artist", con=engine, schema='public', if_exists='replace', index=False)
engine.execute('''
    INSERT INTO public.Artist
    SELECT * FROM temp_artist
    WHERE temp_artist.artist_id NOT IN (SELECT artist_id FROM public.Artist);
''')

Wall time: 47 ms


In [22]:
%%time
# Time Table
cur.execute('''
    CREATE TEMP TABLE IF NOT EXISTS temp_time AS 
    SELECT * FROM Time LIMIT 0;
''')
time_df.to_sql("temp_time", con=engine, schema='public', if_exists='replace', index=False)
engine.execute('''
    INSERT INTO public.Time
    SELECT * FROM temp_time
    WHERE temp_time.start_time NOT IN (SELECT start_time FROM public.Time);
''')

Wall time: 48.8 ms


In [23]:
%%time
# AudioFeatures Table
cur.execute('''
    CREATE TEMP TABLE IF NOT EXISTS temp_audio_feature AS 
    SELECT * FROM AudioFeature LIMIT 0;
''')
audio_features_df.to_sql("temp_audio_feature", con=engine, if_exists='replace', index=False)
engine.execute('''
    INSERT INTO public.AudioFeature
    SELECT * FROM temp_audio_feature
    WHERE temp_audio_feature.song_id NOT IN (SELECT song_id FROM public.AudioFeature);
''')

Wall time: 49.5 ms


In [24]:
%%time
# PlayedSongs Table
cur.execute('''
    CREATE TEMP TABLE IF NOT EXISTS temp_played_song 
    AS SELECT * FROM PlayedSong LIMIT 0;
''')
played_songs_df.to_sql("temp_played_song", con=engine, if_exists='replace', index=False)

engine.execute('''
    INSERT INTO public.PlayedSong
    SELECT * FROM temp_played_song
    WHERE CAST(temp_played_song.played_song_id AS TEXT) NOT IN (SELECT played_song_id FROM public.PlayedSong);
''')

Wall time: 43.5 ms


In [25]:
# Close connection
cur.close()
conn.close()